In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Task 0
Data extraction: get the data from 3 tables & combine it into single `.csv` file.
After that read this file using pandas to create Dataframe.
So it will be all joined data in 1 dataframe. Quick check - should be 74818 rows in it.

In [10]:
import sqlite3


db_path = "../db.sqlite3"
conn = sqlite3.connect(db_path)

query = """
    SELECT oi.order_item_id,
           oi.order_id,
           oi.product_id,
           oi.quantity,
           oi.price AS item_price,
           p.product_name,
           p.category,
           p.price AS product_price,
           o.order_date,
           o.customer_id,
           o.total_amount
    FROM order_items oi
    JOIN products p ON oi.product_id = p.product_id
    JOIN orders o ON oi.order_id = o.order_id
"""

# Читаємо у DataFrame
df = pd.read_sql_query(query, conn)

# Зберігаємо в CSV
df.to_csv("restaurant_full_data.csv", index=False)

# Перевіряємо розмірність
print(df.shape)  # очікується (74818, X)
df.head()


DatabaseError: Execution failed on sql '
        SELECT oi.order_item_id,
               oi.order_id,
               oi.product_id,
               oi.quantity,
               oi.price AS item_price,
               p.product_name,
               p.category,
               p.price  AS product_price,
               o.order_date,
               o.customer_id,
               o.total_amount
        FROM order_items oi
                 JOIN products p ON oi.product_id = p.product_id
                 JOIN orders o ON oi.order_id = o.order_id         ': no such table: order_items

print(os.path.getsize("db.sqlite3"), "байт")# Task 1
Get Top 10 most popular products in restaurant sold by Quantity.
Count how many times each product was sold and create a pie chart with percentage of popularity (by quantity) for top 10 of them.

Example:

![pie chart](../demo/pie.png)

In [ ]:
top_products = (
    df.groupby("product_name")["quantity"]
    .sum()
    .sort_values(ascending=False)
    .head(10)
)

plt.figure(figsize=(8, 8))
plt.pie(
    top_products,
    labels=top_products.index,
    autopct=lambda p: f'{p:.1f}% ({int(p * top_products.sum() / 100)})'
)
plt.title("Top 10 positions in menu by quantity")
plt.ylabel("Quantity")
plt.show()

# Task 2
Calculate `Item Price` (Product Price * Quantity) for each Order Item in dataframe.
And Make the same Top 10 pie chart, but this time by `Item Price`. So this chart should describe not the most popular products by quantity, but which products (top 10) make the most money for restaurant. It should be also with percentage.

In [ ]:
df["item_total"] = df["product_price"] * df["quantity"]

# Групуємо по product_name і сумуємо прибуток
top_revenue_products = (
    df.groupby("product_name")["item_total"]
    .sum()
    .sort_values(ascending=False)
    .head(10)
)

# Будуємо кругову діаграму
plt.figure(figsize=(8, 8))
plt.pie(
    top_revenue_products,
    labels=top_revenue_products.index,
    autopct=lambda p: f'{p:.1f}% (${p * top_revenue_products.sum() / 100:.0f})'
)
plt.title("Top 10 products by total revenue")
plt.ylabel("Revenue")
plt.show()

# Task 3
Calculate `Order Hour` based on `Order Datetime`, which will tell about the specific our the order was created (from 0 to 23). Using `Order Hour` create a bar chart, which will tell the total restaurant income based on the hour order was created. So on x-axis - it will be values from 0 to 23 (hours), on y-axis - it will be the total sum of order prices, which were sold on that hour.

Example:

![bar chart](../demo/bar.png)

In [ ]:
df['order_date'] = pd.to_datetime(df['order_date'])

# Витягуємо годину замовлення
df['Order Hour'] = df['order_date'].dt.hour

# Обчислюємо загальну суму за позицію
df['total_item_price'] = df['item_price'] * df['quantity']

# Групуємо по годині, сумуємо доходи
income_per_hour = df.groupby('Order Hour')['total_item_price'].sum()

# Створюємо діаграму
plt.figure(figsize=(10,6))
income_per_hour.plot(kind='bar', color='skyblue')
plt.title('Total Restaurant Income per Order Hour')
plt.xlabel('Hour of the Day')
plt.ylabel('Total Income')
plt.xticks(rotation=0)  # години горизонтально
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


# Task 4
Make similar bar chart, but right now with `Order Day Of The Week` (from Monday to Sunday), and also analyze total restaurant income by each day of the week.

In [ ]:
df['order_date'] = pd.to_datetime(df['order_date'])

df['Order Day Of The Week'] = df['order_date'].dt.dayofweek

days_map = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday',
            4: 'Friday', 5: 'Saturday', 6: 'Sunday'}

df['Order Day Name'] = df['Order Day Of The Week'].map(days_map)

# Рахуємо повну вартість кожної позиції
df['total_item_price'] = df['item_price'] * df['quantity']

# Групуємо по дню тижня, сумуємо total_item_price
income_per_day = df.groupby('Order Day Name')['total_item_price'].sum()

# Встановлюємо порядок днів тижня
ordered_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
income_per_day = income_per_day.reindex(ordered_days)

plt.figure(figsize=(10,6))
income_per_day.plot(kind='bar', color='coral')
plt.title('Total Restaurant Income per Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Total Income')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()
